# NYT Connections solver

### utils functionality demo

In [2]:
from scripts.puzzle_utils import Puzzle
from scripts.solver import Solver

puzzle = Puzzle()
random_solver = Solver()

# retrieve random puzzle from the database
random_words, id = puzzle.get_random_puzzle()
print(random_words)

# attempt to solve it (the example here is random ordering of words)
random_solver.solve_puzzle(random_words)
attempt_sol = random_solver.build_solution(random_words)
print(attempt_sol)

# check the solution
is_sol, num_corr = puzzle.check_solution(attempt_sol, id)
print(f"Is solution? {is_sol} \nNumber of matches: {num_corr}")

['JERK', 'SAW', 'WRENCH', 'YANK', 'COPY', 'FILE', 'WILLY', 'SAVE', 'TUG', 'BEETHOVEN', 'CHARLOTTE', 'FIND', 'LEVEL', 'BABE', 'HAMMER', 'PRINT']
[{'FIND', 'YANK', 'SAVE', 'PRINT'}, {'SAW', 'BEETHOVEN', 'WILLY', 'HAMMER'}, {'FILE', 'COPY', 'WRENCH', 'BABE'}, {'TUG', 'CHARLOTTE', 'LEVEL', 'JERK'}]
Is solution? False 
Number of matches: 5


In [3]:
from scripts.glove import GloVe

glove = GloVe("embeddings/glove.6B.300d.txt")

Creating GloVe lookup map: 400000it [00:08, 47668.87it/s]


In [4]:
import numpy as np
print(glove.embed_puzzle_words(random_words))

[[-0.019542  0.070949 -0.059923 ... -0.14317   0.39539   0.67533 ]
 [ 0.11598   0.37664  -0.12256  ... -0.012162  0.31691   0.016065]
 [ 0.52188  -0.10094   0.30108  ...  0.5893   -0.49869   0.66253 ]
 ...
 [-0.75336  -0.017923 -0.35947  ... -0.2309    0.81162   0.69205 ]
 [ 0.24715   0.19081  -0.22681  ...  0.24316  -0.086038  0.63675 ]
 [-0.45785   0.2069    0.091824 ...  0.51018   0.87241  -0.14735 ]]


### k-means approach

In [5]:
from scripts.kmeans import KMeansSolver

# retrieve random puzzle from the database
random_words, id = puzzle.get_random_puzzle()
print(random_words)

# attempt to solve it (fixed size kmeans)
kmeans_solver = KMeansSolver(glove)
kmeans_solver.solve_puzzle(random_words)
attempt_sol = kmeans_solver.build_solution(random_words)
print(attempt_sol)

# check the solution
is_sol, num_corr = puzzle.check_solution(attempt_sol, id)
print(f"Is solution? {is_sol} \nNumber of matches: {num_corr}")

['SOLO', 'REAL', 'TANG', 'YUAN', 'SOL', 'POUND', 'TWICE', 'SOLD', 'BUTTER', 'FLUKE', 'ONCE', 'WORLD', 'GOING', 'SOLE', 'MULLET', 'DIVA']
[{'MULLET', 'FLUKE', 'SOL', 'DIVA'}, {'REAL', 'ONCE', 'GOING', 'WORLD'}, {'TWICE', 'BUTTER', 'POUND', 'SOLD'}, {'SOLO', 'YUAN', 'TANG', 'SOLE'}]
Is solution? False 
Number of matches: 6


### neural approach

In [6]:
from scripts.neural.dataset import ConnectionsDataset

dataset = ConnectionsDataset(glove, puzzle)

In [ ]:
from scripts.neural.evaluator import NeuralEvaluator
from scripts.neural.neural_solver import NeuralSolver

evaluator = NeuralEvaluator(1200)
neural_solver = NeuralSolver(puzzle, evaluator, dataset, 10, glove)
neural_solver.train_evaluator()
neural_solver.solve_puzzle(random_words, id)

Training evaluator network: 100%|██████████| 10/10 [00:00<00:00, 215.89it/s]


TypeError: 'Predictor' object is not callable

### testing approaches

In [9]:
from scripts.tester import MethodTester

tester = MethodTester()
results_random = tester.test_solver(random_solver)
results_kmeans = tester.test_solver(kmeans_solver)

print(f"Random Solver:\nNumber of solutions: {np.sum(results_random['is_sol'])}/{len(results_random['is_sol'])}\nAverage number of correct words: {np.mean(results_random['num_corr'])}/16")
# currently not all puzzles are solved with kmeans to do some phrases not having an embeddings
# this mostly happens for complex phrases like 'jack in the box'
# work in progress on that one
print(f"K-means Solver:\nNumber of solutions: {np.sum(results_kmeans['is_sol'])}/{len(results_kmeans['is_sol'])}\nAverage number of correct words: {np.mean(results_kmeans['num_corr'])}/16")

100%|██████████| 645/645 [00:00<00:00, 1754.27it/s]

Random Solver:
Number of solutions: 0/645
Average number of correct words: 5.607751937984496/16
K-means Solver:
Number of solutions: 7/642
Average number of correct words: 7.61214953271028/16
